In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/08 15:54
# @Author  : Wang Yujia
# @File    : basicinfo_GT_models.ipynb
# @Description : 关于GT模型的计算结果的基本信息

# 0. What for
1. 关于GT模型的计算结果的基本信息

# 1. Preparations
## 1.1 全局设置

In [1]:
# P in info_insym
P_1_path = "../data/info_asymm/results/asc_symmetry/GT_asc_symmetry_2_P.csv"
P_2_path = "../data/pt/results/PT_all1288_P.csv"

features_GT = ['product_id','bidincrement','bidfee','retail']

import numpy as np
import csv
import pandas as pd
from visdom import Visdom

# viz = Visdom(env='P',use_incoming_socket=False)

## 1.2 读取data

In [15]:
P_1 = pd.read_csv(P_1_path, encoding="utf-8")
P_2 = pd.read_csv(P_2_path, encoding="utf-8")

# 2. Info about P which is the input of NNs
## 2.1 How many auction settings are included in 'info_insymm', 'GT'

In [12]:
print(f"In 'info_insymm', there is *{P_1.shape}*")
print(f"In 'GT', there is *{P_2.shape}*")

In 'info_insymm', there is *(1, 612)*
In 'GT', there is *(2, 5)*


# 3. Info about dataset
## 3.1 How many ‘settings’ in the dataset
1. dataset指的是samples数量在16之上的所有的ascending-price auctions,GT的两个model用这些data做了generate的过程
2. 根据settings含义的不同，需要有不同的统计

In [13]:
data_path = "../data/info_asymm/datawithnp_asc_symmetry_2_selected.csv"
data = pd.read_csv(data_path, encoding="utf-8")

setting_1 = ['retail','bidfee','bidincrement']
setting_2 = ['product_id','retail','bidfee','bidincrement']

data_1 = data.groupby(setting_1)
data_2 = data.groupby(setting_2)

print(f"按照setting={setting_1}, 一共有 *{len(data_1)}*")
print(f"按照setting={setting_2}, 一共有 *{len(data_2)}*")

按照setting=['retail', 'bidfee', 'bidincrement'], 一共有 *612*
按照setting=['product_id', 'retail', 'bidfee', 'bidincrement'], 一共有 *1303*
另外，虽然两个GT的dataset不同，他们的settings是一样的吗？
按照setting=['retail', 'bidfee', 'bidincrement'], 一共有 *612*
按照setting=['product_id', 'retail', 'bidfee', 'bidincrement'], 一共有 *1303*


## 3.2 Are the dataset in the same size in the amount of setting?
1. 2个GT models用的dataset有微小不同，他们的settings数目是一样的吗: YES

In [ ]:
data_pt_path = "../data/SA_PT/datawithnp_GT_selected.csv"
data_pt = pd.read_csv(data_pt_path, encoding="utf-8")

data_pt_1 = data_pt.groupby(setting_1)
data_pt_2 = data_pt.groupby(setting_2)

print(f"按照setting={setting_1}, 一共有 *{len(data_pt_1)}*")
print(f"按照setting={setting_2}, 一共有 *{len(data_pt_2)}*")